<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigdataII_06_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# いつものDriveマウント

from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/xxx'

Matplotlib文字化け対策

In [ ]:
x = range(6)
y = np.linspace(0,5,6)

plt.plot(x, y)
plt.title("テスト")
plt.show()

In [ ]:
# 「IPA」フォントをインストール
!apt-get -y install fonts-ipafont-gothic
#!rm /root/.cache/matplotlib/fontlist-v300.json # キャッシュを削除

!pip install japanize-matplotlib
import japanize_matplotlib #matplotlib日本語化
import seaborn as sns
sns.set(font="IPAexGothic") #日本語フォント設定

In [ ]:
# タイトル確認
plt.plot(x, y)
plt.title("テスト")
plt.show()

# 主成分分析（PCA） 演習

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 基本的な使い方

### データ生成

In [ ]:
# ランダムにデータを生成
rng = np.random.RandomState(1)
X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T # 分散させるために2x2の行列を掛ける

In [ ]:
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal'); # 軸の比率を同じにする

### PCA, 主成分と共分散を取得する

In [ ]:
from sklearn.decomposition import PCA

### here ###
pca =

In [ ]:
print('主成分\n', pca.components_)
print('平均\n', pca.mean_)
print('分散\n', pca.explained_variance_)
print('共分散\n', pca.get_covariance())

### 平均と共分散を直接計算して、比較してみる。

In [ ]:
mu = np.mean(X, axis=0)

z = X - mu

cv = np.cov(z[:,0], z[:,1], bias=1)

print('平均\n', mu)
print('共分散\n', cv)

### 補足：

共分散行列の固有値と固有ベクトルを計算する。
共分散行列の固有ベクトルは主成分と一致する。

※データによっては180度反転してる場合もある

In [ ]:
W, v = np.linalg.eig(cv)

print('固有値\n', W)
print('固有ベクトル\n', v)

ベクトルを描画する関数

> 参考：矢印を描画する　https://qiita.com/haru1843/items/e85fe45163cd9763023c

> 参考：for loopで便利な zip, enumerate　https://python.civic-apps.com/zip-enumerate/

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0,
                    color="red")
    ax.annotate('', v1, v0, arrowprops=arrowprops)

### オリジナルデータのプロットと、主成分を軸にしたプロット

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

# オリジナルデータをプロット
ax[0].scatter(X[:, 0], X[:, 1], alpha=0.5)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v, ax=ax[0])
ax[0].axis('equal');
ax[0].set(
          xlabel='x', ylabel='y',
          title='input')

# 平均 0、分散 1にする : whitening
### here ###
pca =


# 主成分（PC1, PC2）空間へのプロット
### here ###
X_pca = 


ax[1].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
draw_vector([0, 0], [0, 3], ax=ax[1])
draw_vector([0, 0], [3, 0], ax=ax[1])
ax[1].axis('equal')
ax[1].set(
        xlabel='component 1', ylabel='component 2',
        title='principal components',
        xlim=(-5, 5), ylim=(-3, 3.1));

### 次元削減

In [ ]:
# 2次元 ⇒ 1次元
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

In [ ]:
# PC1のプロット
X_new = pca.inverse_transform(X_pca)
plt.scatter(X[:, 0], X[:, 1], alpha=0.3)
plt.scatter(X_new[:, 0], X_new[:, 1], color='red')
plt.axis('equal');

# 例：Wine データセット

178行のワインサンプルと、それらの科学的性質を表す13列の特徴量で構成されているWineデータセットを読み込む。

In [ ]:
import pandas as pd
from sklearn.datasets import load_wine

wine = load_wine()  # Wineデータセットの読み込み
df_wine = pd.DataFrame(wine.data, columns=wine.feature_names)
df_wine['class'] = wine.target
df_wine

データの標準化（平均０、分散１）

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df_wine.iloc[:, :-1].values  # classカラム以外を取得
y = df_wine.iloc[:, -1].values  # classカラムを取得
# 標準化
sc = StandardScaler()
X_std = sc.fit_transform(X)
X_std

PCA 直接計算してみる

In [ ]:
import numpy as np

# 分散共分散行列の作成
cov_mat = np.cov(X_std.T)
# 分散共分散行列の固有値、固有ベクトルを作成
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
# 固有値、固有ベクトルのペアを作成
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:,i]) for i in range(len(eigen_vals))]
# 上記のペアを固有値の大きい順にソート
eigen_pairs.sort(key=lambda k: k[0], reverse=True)

w1 = eigen_pairs[0][1]  # 第1主成分に対応する固有ベクトル
w2 = eigen_pairs[1][1]  # 第2主成分に対応する固有ベクトル

# 射影行列の作成
W = np.stack([w1, w2], axis=1)
# 次元圧縮 (13次元 -> 2次元)
X_pca = X_std @ W

PCAライブラリ実行

In [ ]:
# PCAライブラリをインポート
from sklearn.decomposition import PCA

# 次元圧縮 (13次元 -> 2次元)
X_pca = PCA(n_components=2, random_state=42).fit_transform(X_std)  # n_componentsは圧縮後の次元数

主成分（PC1、PC2）2次元空間へのプロット

In [ ]:
import matplotlib.pyplot as plt

colors = ['#de3838', '#007bc3', '#ffd12a']  # 緋色、露草色、山吹色 
markers = ['o', 'x', ',']
for l, c, m, in zip(np.unique(y), colors, markers):
    plt.scatter(X_pca[y==l, 0], X_pca[y==l, 1],
                c=c, marker=m, label=l)

plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend()
plt.show()

# 応用例：プロ野球選手50名分のデータ

In [ ]:
import pandas as pd
url ='baseball.csv'
raw_data = pd.read_csv(
    url,
    thousands  = ',',
    encoding='utf-8'
)

raw_data.head()

数値データの標準化
StandardScalerを用いてデータを標準化

In [ ]:
from sklearn.preprocessing import StandardScaler
raw_data_drop = raw_data.drop(columns=['選手名','順位', 'チーム'] ) # 文字が入るとPCAできないのでcolumnの削除

# StandardScalerを用いたdatasetの標準化
scaler = StandardScaler()
scaler.fit(raw_data_drop)
scaler.transform(raw_data_drop)
raw_data_drop_normalize = pd.DataFrame(scaler.transform(raw_data_drop), columns=raw_data_drop.columns) # 標準化したdatasetをraw_data_drop_normalizeとする

raw_data_drop_normalize.head()


PCA 実行（22次元 ⇒ 3次元）

In [ ]:
from sklearn.decomposition import PCA
# raw_data_drop_normalizeを用いたPCA
pca = PCA(n_components = 3) #3軸でPCA classを作成する
pca.fit(raw_data_drop_normalize) #fitting、実際のPCA処理はここ

# PCAの結果はarrayで返ってくるのでをData.Frameにする
pca_result = pca.transform(raw_data_drop_normalize)
pca_result = pd.DataFrame(pca_result)
pca_result.columns = ['PC1', 'PC2', 'PC3'] 

pca_result.head()

クラスタリング

In [ ]:
from sklearn.cluster import KMeans
#クラスタリング
num_cluster = 4
color = ["red", "blue", "green", "orange"] # クラスタリング用の色

pca_result_cluster = KMeans(n_clusters=num_cluster).fit(pca_result)

pca_result_cluster.labels_

In [ ]:
player_name = raw_data['選手名'] # annotation用に格納
pca_result["name"] = player_name
pca_result["cluster"] = pca_result_cluster.labels_
pca_result.head()

3D 描画

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from PIL import Image # アニメーション用
from matplotlib import animation # アニメーション用
from io import BytesIO # アニメーション用

In [ ]:
class Annotation3D(Annotation):
    '''Annotate the point xyz with text s'''

    def __init__(self, s, xyz, *args, **kwargs):
        Annotation.__init__(self,s, xy=(0,0), *args, **kwargs)
        self._verts3d = xyz

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.xy=(xs,ys)
        Annotation.draw(self, renderer)

# annotate3Dを定義
def annotate3D(ax, s, *args, **kwargs):
    '''add anotation text s to to Axes3d ax'''

    tag = Annotation3D(s, *args, **kwargs)
    ax.add_artist(tag)

# plot_3Dを定義


In [ ]:
def plot_3D(data, angle = 50):
    # 3d plot用Figure
    fig = plt.figure(num=None, figsize=(12, 12), dpi=72)
    ax = fig.gca(projection = '3d')
    for i in range(len(data.index)):
        ax.scatter3D(data.iloc[i,0], data.iloc[i,1],data.iloc[i,2], c=color[int(data.iloc[i,4])]) # プロットの座標を指定
        annotate3D(ax, s=str(data.iloc[i,3]), xyz=(data.iloc[i,0], data.iloc[i,1],data.iloc[i,2]),
        fontsize=12,
        xytext=(-3,3),
        textcoords='offset points', ha='right',va='bottom', fontname="IPAexGothic") # annotation
    ax.view_init(30, angle)
    ax.set_xlim(data.describe().at['min', 'PC1'], data.describe().at['max', 'PC1'])
    ax.set_ylim(data.describe().at['min', 'PC2'], data.describe().at['max', 'PC2'])
    ax.set_zlim(data.describe().at['min', 'PC3'], data.describe().at['max', 'PC3'])
    ax.set_xlabel('PC1 ' + str(round(pca.explained_variance_ratio_[0]*100, 1)) + "%") # 軸に寄与率を表示
    ax.set_ylabel('PC2 ' + str(round(pca.explained_variance_ratio_[1]*100, 1)) + "%")
    ax.set_zlabel('PC3 ' + str(round(pca.explained_variance_ratio_[2]*100, 1)) + "%")
    buf = BytesIO()
    fig.savefig(buf, bbox_inches='tight', pad_inches=0.0)
    return Image.open(buf)

plot_3D(pca_result)
plt.show()

In [ ]:
# gif animationを作る場合
# images = [plot_3D(pca_result,angle) for angle in range(180)]
# images[0].save('dataset_temp/output.gif', save_all=True, append_images=images[1:], duration=100, loop=0)

# Plotly

https://plotly.com/python/

https://plotly.com/python/3d-scatter-plots/

https://data-analytics.fun/2021/06/08/plotly-data-visualization-1/



In [ ]:
# -*- coding: utf-8 -*-
import plotly.express as px
import pandas as pd

df = pd.DataFrame(pca_result.loc[:,['PC1','PC2','PC3','name','cluster']])

# データフレームのデータを描画する
fig = px.scatter_3d(df, x='PC1', y='PC2', z='PC3', hover_name="name", color='cluster')
fig.show()